In [1]:
import alpaca_trade_api as tradeapi
import numpy as np
from scipy import optimize as opt
import pandas as pd
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
from bs4 import BeautifulSoup
import requests
from sklearn.linear_model import LinearRegression
import datetime 
import workdays as wkdy

/mnt/c/Users/natha/Documents/TourdeFinance/trading_env/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
with open("keys.txt") as keys_file :
    GENERAL_API_KEY = keys_file.readline().strip()
    SECRET_API_KEY = keys_file.readline().strip()
ALPACA_URL_BASE  = "https://paper-api.alpaca.markets"
ALPACA_DATA_URL_BASE = "https://data.alpaca.markets/v1"

In [3]:
api  = tradeapi.REST(GENERAL_API_KEY, SECRET_API_KEY, ALPACA_URL_BASE, api_version = 'v2')
account = api.get_account()

In [4]:
#use get_available_datasets() to see all the different Fama French datasets available
ff5 = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench')
ff5 = ff5[0][::-1] #change it from a dict to a dataframe and then change it 
                   #so it's from newest to oldest
ff5.reset_index(inplace=True)

In [5]:
ff5

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2020-02-28,-0.78,0.06,-0.78,0.32,-0.41,0.006
1,2020-02-27,-4.22,0.69,0.14,-0.35,-0.09,0.006
2,2020-02-26,-0.52,-0.79,-1.26,-0.52,-0.12,0.006
3,2020-02-25,-3.09,-0.34,-0.72,-0.63,0.03,0.006
4,2020-02-24,-3.38,0.15,-0.04,-0.37,0.20,0.006
...,...,...,...,...,...,...,...
1225,2015-04-17,-1.23,-0.43,0.23,0.13,0.10,0.000
1226,2015-04-16,-0.08,-0.10,-0.20,-0.23,-0.21,0.000
1227,2015-04-15,0.57,0.27,0.32,-0.19,-0.07,0.000
1228,2015-04-14,0.11,-0.13,0.19,0.09,0.09,0.000


In [53]:
# get sp500 tickers, code courtesy of Will Everett
page = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
soup = BeautifulSoup(page.content, 'html.parser')
sp500_tickers = [row.find('td').get_text().replace('\n','')
                 for row in soup.find('tbody').find_all('tr')[1:]]
sp500_tickers.remove('BRK.B')
sp500_tickers.remove('BF.B')
sp500_tickers.sort()

In [54]:
days_of_data = 1000
open_close = pd.DataFrame()
for i in range(0,len(sp500_tickers),100) :
    data = api.get_barset(sp500_tickers[i:i+99],'day',limit=days_of_data)
    for ticker in sp500_tickers[i:i+99] :
        opin = []
        close = []
        for day in range(days_of_data - 1, -1, -1) :
            try :
                if (data[ticker][day].o == 0) :
                    opin.append(np.nan)
                else :
                    opin.append(data[ticker][day].o)
                close.append(data[ticker][day].c)
            except :
                opin.append(np.nan)
                close.append(np.nan)
        open_close[ticker + ' Open'] = opin
        open_close[ticker + ' Close'] = close

In [55]:
for tkr in sp500_tickers :
    try :
        open_close[tkr + ' Open'][0]
    except :
        sp500_tickers.remove(tkr)

In [56]:
#number of days between today and when the stock market got hit by COVID-19
today = datetime.date.today()
days_since_crash = wkdy.networkdays(datetime.date(2020, 2, 19), today)
days_since_crash

38

In [58]:
hit_hard = []
for tkr in sp500_tickers :
    dif = (open_close[tkr + ' Open'][days_since_crash] -
           open_close[tkr + ' Close'][0]) / open_close[tkr + ' Open'][days_since_crash]
    hit_hard.append(dif)

In [76]:
#finds the n greatest numbers in a list
def n_max(lis, n) :
    maxes = pd.DataFrame()
    indexes = list(range(0,len(lis)))
    maxes['List'] = lis
    maxes['Indexes'] = indexes
    maxes = maxes.sort_values(['List'], axis = 1, ascending = False)
    return maxes['Indexes']

In [30]:
backup = sp500_tickers
buy = n_max(sp500_tickers, 15)

480

In [77]:
a = [1,2,3,4,5,6,7,8,9]
maxes = n_max(a, 4)
maxes

KeyError: 'List'

In [36]:
dailyreturns = pd.DataFrame()
for tkr in sp500_tickers[0:99] :
    dailyreturns_col = []
    for day in range(days_of_data - 1, -1, -1) :
        if(data1[tkr][day].o == 0) :
            dailyreturns_col.append(np.nan)
        else :
            dailyreturns_col.append((data1[tkr][day].c - 
                             data1[tkr][day].o)/
                             data1[tkr][day].o)
    dailyreturns[tkr] = dailyreturns_col
for tkr in sp500_tickers[100:199] :
    dailyreturns_col = []
    for day in range(days_of_data - 1, -1, -1) :
        if(data2[tkr][day].o == 0) :
            dailyreturns_col.append(np.nan)
        else :
            dailyreturns_col.append((data2[tkr][day].c - 
                             data2[tkr][day].o)/
                             data2[tkr][day].o)   
    dailyreturns[tkr] = dailyreturns_col
for tkr in sp500_tickers[200:299] :
    dailyreturns_col = []
    for day in range(days_of_data - 1, -1, -1) :
        if(data3[tkr][day].o == 0) :
            dailyreturns_col.append(np.nan)
        else :
            dailyreturns_col.append((data3[tkr][day].c - 
                             data3[tkr][day].o)/
                             data3[tkr][day].o)   
    dailyreturns[tkr] = dailyreturns_col
for tkr in sp500_tickers[300:399] :
    dailyreturns_col = []
    for day in range(days_of_data - 1, -1, -1) :
        if(data4[tkr][day].o == 0) :
            dailyreturns_col.append(np.nan)
        else :
            dailyreturns_col.append((data4[tkr][day].c - 
                             data4[tkr][day].o)/
                             data4[tkr][day].o)   
    dailyreturns[tkr] = dailyreturns_col
for tkr in sp500_tickers[400:499] :
    dailyreturns_col = []
    for day in range(days_of_data - 1, -1, -1) :
        if(data5[tkr][day].o == 0) :
            dailyreturns_col.append(np.nan)
        else :
            dailyreturns_col.append((data5[tkr][day].c - 
                             data5[tkr][day].o)/
                             data5[tkr][day].o)   
    dailyreturns[tkr] = dailyreturns_col

IndexError: list index out of range

In [10]:
dailyreturns

,LHO,NVDA,VKTX,COG,KO
0,-0.013573,-0.033699,0.012421,-0.004932,-0.009402
1,0.004309,0.051555,0.024115,0.045137,0.026856
2,-0.022778,-0.039917,-0.020313,0.018144,0.005943
3,-0.017887,0.046385,0.022500,-0.011893,0.040715
4,0.015424,-0.049447,0.002069,-0.013537,-0.008263
...,...,...,...,...,...
895,0.030838,0.036099,-0.027273,-0.001035,0.008189
896,-0.020466,0.006399,-0.033824,-0.019300,-0.004164
897,0.007346,-0.011875,-0.018737,0.001698,-0.011597
898,-0.014075,0.034293,0.010769,-0.005950,0.019452


In [11]:
#calculate the days between now and the fama french data
today = datetime.date.today()
ff5end_date = ff5['Date'][0]
#presidentsday = [datetime.date(2020, 2, 17)]
days_offset = wkdy.networkdays(ff5end_date.date(), today)

In [13]:
num_data = days_of_data - days_offset

872

In [24]:
X = pd.DataFrame(ff5[['Mkt-RF','SMB','HML']][0:num_data], columns=['Mkt-RF','SMB','HML'])
y = dailyreturns['KO'][days_offset:]

In [25]:
model = LinearRegression().fit(X, y)
betas = model.coef_
betas

array([-3.33648530e-04,  5.02798968e-04, -2.19584049e-05])

In [69]:

weights = np.ones(len(ff5))
i = 1
for j in range(len(ff5)) :
    weights[j] *= i
    i -= .00005
weights

array([1.     , 0.99995, 0.9999 , ..., 0.9384 , 0.93835, 0.9383 ])

In [4]:
factors = ['Mkt-RF','SMB','HML','RMW','CMA']
premiums = []
for factor in factors :
    av = np.average(ff5[factor])
    premiums.append(av)
premiums

[0.035162337662337666, -0.015673701298701298, -0.021566558441558442]

In [28]:
expected_returns = ff5['RF'][0]
for i in range(3) :
    expected_returns += betas[i]*premiums[i]
expected_returns

0.005980860984122728

In [3]:
#c of objective funct
#A_ub and b_ub
num_assets = 5
elem_vect = np.ones(num_assets)
#solution = opt.linprog(A_eq = elem_vect, b_eq = np.ones(1))

In [6]:
api.submit_order(symbol = 'AAPL',
                 qty = 3,
                 side = 'buy',
                 type = 'market',
                 time_in_force = 'day')

Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': 'a192e8d2-b3b1-4a79-8054-dfd649e61a56',
    'created_at': '2020-04-03T21:32:04.784938Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '77439e17-4f58-401c-a38c-89029d04868c',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '3',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-03T21:32:04.777419Z',
    'symbol': 'AAPL',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-03T21:32:04.784938Z'})

In [ ]:
clock = api.get_clock()
if clock.is_open :
    #trade away! 

In [7]:
ff5

{0:             Mkt-RF   SMB   HML   RMW   CMA     RF
 Date                                             
 2015-03-09    0.37  0.09 -0.02  0.26 -0.03  0.000
 2015-03-10   -1.63  0.43 -0.47  0.09 -0.14  0.000
 2015-03-11   -0.04  0.62  0.52 -0.47 -0.03  0.000
 2015-03-12    1.28  0.40  0.50 -0.06  0.23  0.000
 2015-03-13   -0.57  0.19 -0.04 -0.10 -0.13  0.000
 ...            ...   ...   ...   ...   ...    ...
 2020-01-27   -1.56  0.27 -0.48 -0.59  0.05  0.006
 2020-01-28    1.02 -0.15 -0.42  0.17 -0.10  0.006
 2020-01-29   -0.10 -0.44 -0.95  0.25  0.19  0.006
 2020-01-30    0.34 -0.64  0.67 -0.39 -0.11  0.006
 2020-01-31   -1.74 -0.52 -0.34 -0.54 -0.69  0.006
 
 [1235 rows x 6 columns],
 'DESCR': 'F-F Research Data 5 Factors 2x3 daily\n-------------------------------------\n\nThis file was created by CMPT_ME_BEME_OP_INV_RETS_DAILY using the 202001 CRSP database. The 1-month TBill return is from Ibbotson and Associates, Inc.\n\n  0 : (1235 rows x 6 cols)'}